In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from binance.client import Client
import pandas as pd
import time

# Initialize Binance client (no API key needed for public data)
client = Client()

# Define your tickers - Binance uses different symbols, typically without '-USD'
tickers = ["BTCUSDT", "XRPUSDT", "ETHUSDT", "SOLUSDT", "DOGEUSDT", "ADAUSDT"]
# , 
#            "SHIBUSDT", "DOTUSDT", "BTTUSDT", "LINKUSDT", "ALGOUSDT", "AVAXUSDT",
#            "XLMUSDT", "NEARUSDT", "LTCUSDT", "CHZUSDT", "POLUSDT", "GRTUSDT",
#            "LRCUSDT", "ARBUSDT", "UNIUSDT", "GALAUSDT", "INJUSDT", "TRXUSDT", "CRVUSDT",
#            "ANKRUSDT", "NMRUSDT", "WOOUSDT", "MANAUSDT", "AAVEUSDT", "QNTUSDT", "BCHUSDT",
#            "SUSHIUSDT", "APEUSDT", "ZRXUSDT", "ETCUSDT", "KSMUSDT", "SANDUSDT", "IMXUSDT", 
#            "1INCHUSDT", "OPUSDT", "ATOMUSDT", "POWRUSDT", "AXSUSDT", "YFIUSDT", 
#            "SNXUSDT", "MKRUSDT", "STORJUSDT", "GNOUSDT", "BATUSDT", "REQUSDT", "COMPUSDT", 
#            "XTZUSDT", "BNTUSDT", "ENJUSDT", "EOSUSDT"]

# Date range
start_str = "2023-01-01"
end_str = "2024-01-01"

# Helper to get historical data from Binance
def get_binance_ohlcv(symbol, interval, start_str, end_str):
    df = pd.DataFrame(client.get_historical_klines(symbol, interval, start_str, end_str))
    if df.empty:
        return None
    df = df.iloc[:, 0:6]
    df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df.astype(float)
    return df

# Create a dictionary of DataFrames for each ticker
data = {}
for ticker in tickers:
    try:
        print(f"Downloading data for {ticker}...")
        df = get_binance_ohlcv(ticker, Client.KLINE_INTERVAL_1DAY, start_str, end_str)
        if df is not None:
            data[ticker] = df
        time.sleep(0.3)  # To avoid hitting Binance rate limits
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")

In [ ]:
for ticker, df in data.items():
    # Calculate close-to-close change
    df['close_change'] = df['close'].pct_change()
    
    # Count positive changes in the last 28 days (rolling window)
    df['pos_changes_30d'] = df['close_change'].gt(0).rolling(window=30, min_periods=30).sum()

    df["change_class"] = pd.cut(df["pos_changes_30d"], bins=[0, 6, 12, 18, 24, 30], labels=[0, 1, 2, 3, 4])

In [ ]:
data["ADAUSDT"]

In [ ]:
import pandas as pd

# Combine all tickers into one DataFrame for easier class processing
all_df = pd.concat(data.values(), keys=data.keys(), names=['ticker', 'timestamp'])

# Add direction and magnitude columns
all_df['change_direction'] = (all_df['close_change'] > 0).astype(int)
all_df['change_magnitude_pos'] = all_df['close_change'].clip(lower=0)
all_df['change_magnitude_neg'] = all_df['close_change'].clip(upper=0).abs()


In [ ]:
import torch
import pyro
import pyro.distributions as dist

def stock_model(change_direction, mag_pos, mag_neg):
    N = change_direction.shape[0]

    # Priors
    p = pyro.sample("p", dist.Beta(1, 1))
    mu_pos = pyro.sample("mu_pos", dist.Normal(0, 10))
    sigma_pos = pyro.sample("sigma_pos", dist.HalfCauchy(5))
    mu_neg = pyro.sample("mu_neg", dist.Normal(0, 10))
    sigma_neg = pyro.sample("sigma_neg", dist.HalfCauchy(5))

    with pyro.plate("data", N):
        # Direction likelihood
        pyro.sample("dir", dist.Bernoulli(p), obs=change_direction)
        # Positive magnitude likelihood (only where direction is 1)
        pos_idx = (change_direction == 1)
        if pos_idx.any():
            pyro.sample("mag_pos", dist.Normal(mu_pos, sigma_pos).expand([N]).mask(pos_idx), obs=mag_pos)
        # Negative magnitude likelihood (only where direction is 0)
        neg_idx = (change_direction == 0)
        if neg_idx.any():
            pyro.sample("mag_neg", dist.Normal(mu_neg, sigma_neg).expand([N]).mask(neg_idx), obs=mag_neg)


In [ ]:
all_df

In [ ]:
import pyro.infer.mcmc as mcmc
import pyro.infer
import numpy as np

results = {}
class_samples = {}

for class_label in sorted(all_df['change_class'].dropna().unique()):
    print(f"\nRunning MCMC for class {class_label}...")
    class_df = all_df[all_df['change_class'] == class_label].dropna(subset=['close_change'])

    # Prepare tensors
    change_direction = torch.tensor(class_df['change_direction'].values, dtype=torch.float32)
    mag_pos = torch.tensor(class_df['change_magnitude_pos'].values, dtype=torch.float32)
    mag_neg = torch.tensor(class_df['change_magnitude_neg'].values, dtype=torch.float32)

    # Run MCMC using NUTS
    nuts_kernel = mcmc.NUTS(stock_model)
    mcmc_run = mcmc.MCMC(nuts_kernel, num_samples=500, warmup_steps=200, num_chains=1)
    mcmc_run.run(change_direction, mag_pos, mag_neg)

    # Extract posterior means for parameters
    samples = mcmc_run.get_samples()
    param_means = {k: v.mean().item() for k, v in samples.items()}
    results[class_label] = param_means
    class_samples[class_label] = samples

    print(f"Class {class_label} parameter means: {param_means}")

print("\nLearned posterior means by class:")
for c, params in results.items():
    print(f"Class {c}: {params}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
signal_edges = {}

for c in (1,2,3):
    # Example: assuming mcmc_run is your MCMC object after .run()
    posterior_samples = class_samples[c]
    num_predictive_samples = 1000

    # Randomly select 1000 samples (if you have more)
    idx = np.random.choice(len(posterior_samples['p']), num_predictive_samples, replace=True)
    samples = {k: v[idx] for k, v in posterior_samples.items()}

    import torch

    future_days = 5
    predicted_changes = torch.zeros((num_predictive_samples, future_days))

    for i in range(num_predictive_samples):
        p = samples['p'][i]
        mu_pos = samples['mu_pos'][i]
        sigma_pos = samples['sigma_pos'][i]
        mu_neg = samples['mu_neg'][i]
        sigma_neg = samples['sigma_neg'][i]

        for d in range(future_days):
            direction = torch.bernoulli(p)
            if direction == 1:
                change = torch.normal(mu_pos, sigma_pos)
            else:
                change = -torch.abs(torch.normal(mu_neg, sigma_neg))  # negative change
            predicted_changes[i, d] = change



    # Cumulative sum along the days axis for each sample
    random_walks = predicted_changes.cumsum(dim=1).numpy()  # shape: (num_samples, 5)

    # plt.figure(figsize=(10, 6))
    # for i in range(10):  # Plot 10 random paths
    #     plt.plot(range(1, 6), random_walks[i], alpha=0.7)
    # plt.xlabel('Day')
    # plt.ylabel('Cumulative % Change')
    # plt.title('Random Walk Simulations for Next 5 Days')
    # plt.grid(True)
    # plt.show()

    mean_walk = random_walks.mean(axis=0)
    lower_walk = np.quantile(random_walks, 0.2, axis=0)
    upper_walk = np.quantile(random_walks, 0.8, axis=0)
    signal_edges[c] = {"lower": lower_walk, "upper": upper_walk, "mean": mean_walk}

    plt.figure(figsize=(10, 6))
    plt.plot(range(1, 6), mean_walk, label=f'Mean Random Walk class {c}', color='blue')
    plt.fill_between(range(1, 6), lower_walk, upper_walk, color='blue', alpha=0.2, label='Credible Interval')
    plt.xlabel('Day')
    plt.ylabel('Cumulative % Change')
    plt.title('Mean and 95% Credible Interval: 5-Day Random Walk')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
def apply_signal(df, signal_edges):
    if pd.isnull(df["change_class"]):
        return None
    
    lower = signal_edges[df["change_class"]]["lower"]
    upper = signal_edges[df["change_class"]]["upper"]
    mean = signal_edges[df["change_class"]]["mean"]
    
    # if df["rolling_5_compound"] > upper[4] and df["return-5"] > mean[4] and df["return-4"] > mean[3] and df["return-3"] > mean[2] and df["return-2"] > mean[1] and df["return-1"] > mean[0]:
    #     return 1
    if df["rolling_3_compound"] > upper[2] and df["return-3"] > mean[2] and df["return-1"] > mean[1] and df["return-1"] > mean[0]:
        return df["change_class"]

    return None

In [ ]:
for tickr in tickers:
    data[tickr]["rolling_1_compound"] = data[tickr]["close_change"].rolling(window=1).apply(lambda x: np.prod(1 + x) - 1)
    data[tickr]["rolling_2_compound"] = data[tickr]["close_change"].rolling(window=2).apply(lambda x: np.prod(1 + x) - 1)
    data[tickr]["rolling_3_compound"] = data[tickr]["close_change"].rolling(window=3).apply(lambda x: np.prod(1 + x) - 1)
    data[tickr]["rolling_4_compound"] = data[tickr]["close_change"].rolling(window=4).apply(lambda x: np.prod(1 + x) - 1)
    data[tickr]["rolling_5_compound"] = data[tickr]["close_change"].rolling(window=5).apply(lambda x: np.prod(1 + x) - 1)
    
    data[tickr]["return-5"] = data[tickr]["close_change"].shift(5)
    data[tickr]["return-4"] = data[tickr]["close_change"].shift(4)
    data[tickr]["return-3"] = data[tickr]["close_change"].shift(3)
    data[tickr]["return-2"] = data[tickr]["close_change"].shift(2)
    data[tickr]["return-1"] = data[tickr]["close_change"].shift(1)

    data[tickr]["signal"] = data[tickr].apply(apply_signal, args=(signal_edges,), axis=1)

    # Plot
    plt.figure(figsize=(12,6))
    plt.plot(data[tickr].index, data[tickr]['close'], label=f'{tickr} Price')
    plt.scatter(data[tickr].loc[data[tickr]['signal'] == 1].index, data[tickr].loc[data[tickr]['signal'] == 1]['close'], color='g', label='Buy Signal 1')
    plt.scatter(data[tickr].loc[data[tickr]['signal'] == 2].index, data[tickr].loc[data[tickr]['signal'] == 2]['close'], color='r', label='Buy Signal 2')
    plt.scatter(data[tickr].loc[data[tickr]['signal'] == 3].index, data[tickr].loc[data[tickr]['signal'] == 3]['close'], color='b', label='Buy Signal 3')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.title(f'{tickr} Course (Including Compounding Effects)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()